In [70]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

from sklearn import set_config
set_config(transform_output='pandas')

# Загрузка данных

In [71]:
# %%capture
# !wget https://www.dropbox.com/s/64ol9q9ssggz6f1/data_ford_price.xlsx

In [72]:
data = pd.read_excel('data/data_ford_price.xlsx', engine='openpyxl')

## Предобработка данных
Давайте посмотрим, как StandardScaler, MinMax и RobustScaler справятся с нашими данными. 

Для наглядности будем использовать три столбца.

In [73]:
data = data[['price', 'year', 'weather']]
data.dropna(inplace=True)
data_scaled = data.copy()

In [74]:
data_scaled

,price,year,weather
0,43900,2016,59.0
1,15490,2009,52.0
2,2495,2002,45.0
3,1300,2000,49.0
5,6995,2003,50.0
...,...,...,...
7012,22500,2015,59.0
7013,5975,2005,50.0
7014,9999,2006,59.0
7015,10900,2011,47.0


## Масштабирование

In [75]:
scaler = StandardScaler()

# Отобранные признаки трансформирую и сразу заменяю значения в исходном датафрейме
col_names = ['price', 'weather']
data_scaled[col_names] = scaler.fit_transform(data_scaled[col_names])
data_scaled

,price,year,weather
0,2.558728,2016,0.862068
1,0.067373,2009,-0.017965
2,-1.072197,2002,-0.897998
3,-1.176990,2000,-0.395122
5,-0.677579,2003,-0.269403
...,...,...,...
7012,0.682100,2015,0.862068
7013,-0.767025,2005,-0.269403
7014,-0.414149,2006,0.862068
7015,-0.335138,2011,-0.646560


In [76]:
X, y = data_scaled.drop(columns='year'), data_scaled['year']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=40)

In [77]:
model = LinearRegression()
model.fit(X_train, y_train)

y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

mae = mean_absolute_error(y_train, y_train_pred)
print(f'Train MAE: {mae:.3f}')
 
mae = mean_absolute_error(y_test, y_test_pred)
print(f'Test MAE: {mae:.3f}')

Train MAE: 3.177
Test MAE: 3.041


### Использование разных скейлеров

In [78]:
# Создаю словарь скалеров
scalers = {
    'Standart': StandardScaler(),
    'MinMax': MinMaxScaler(),
    'Robust': RobustScaler()
}

result = {} # пустой словарь для результатов

In [79]:
for name, scaler in scalers.items(): # итеорируюсь по словарю скалеров
    data_scaled = data.copy()
    
    col_names = ['price', 'weather']
    data_scaled[col_names] = scaler.fit_transform(data_scaled[col_names])
    
    X, y = data_scaled.drop(columns='year'), data_scaled['year']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=40)
    
    model = LinearRegression()
    model.fit(X_train, y_train)
    
    y_test_pred = model.predict(X_test)
    
    # Для тествой выборки сохраняю результаты по каждому из скалеров
    mae = mean_absolute_error(y_test, y_test_pred)
    result[name] = round(mae, 3)

In [80]:
result

{'Standart': 3.041, 'MinMax': 3.041, 'Robust': 3.041}

### Задание 5.2
Отмасштабируйте признак price с помощью RobustScaler.

В качестве ответа запишите результат выполнения кода data['price'][1]. 

Ответ округлите до двух знаков после точки-разделителя.

In [61]:
rs = RobustScaler()

In [62]:
data['price'] = rs.fit_transform(data[['price']])

In [66]:
# Просто значение цены после скалинга из строки с индексом 1
round(data['price'][1], 2)

0.19